<a href="https://colab.research.google.com/github/diaboloshogunate/GoogleColabML/blob/main/fashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor

In [47]:
# parameters
batch_size = 4
learning_rate = 0.001
momentum = 0
epochs = 2
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
layers = [nn.Linear(28*28, 1024), nn.ReLU(), nn.Linear(1024, 10)]
nn.Sequential(*layers)

Sequential(
  (0): Linear(in_features=784, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=10, bias=True)
)

In [48]:
# datasets
trainset = torchvision.datasets.FashionMNIST('./data', download=True, train=True, transform=ToTensor())
testset = torchvision.datasets.FashionMNIST('./data', download=True, train=False, transform=ToTensor())

In [49]:
# data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
shuffle=False, num_workers=2)

In [50]:
# classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [51]:
for X, y in testloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([4, 1, 28, 28])
Shape of y:  torch.Size([4]) torch.int64


In [52]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [53]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, stack):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = stack

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(stack).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [54]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [55]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.279865  [    0/60000]
loss: 2.354511  [  400/60000]
loss: 2.084656  [  800/60000]
loss: 1.998106  [ 1200/60000]
loss: 1.925862  [ 1600/60000]
loss: 1.798301  [ 2000/60000]
loss: 1.648045  [ 2400/60000]
loss: 1.798101  [ 2800/60000]
loss: 1.808859  [ 3200/60000]
loss: 1.615044  [ 3600/60000]
loss: 1.423078  [ 4000/60000]
loss: 1.444520  [ 4400/60000]
loss: 1.825121  [ 4800/60000]
loss: 1.321168  [ 5200/60000]
loss: 1.524265  [ 5600/60000]
loss: 1.627309  [ 6000/60000]
loss: 1.393851  [ 6400/60000]
loss: 1.548409  [ 6800/60000]
loss: 1.302371  [ 7200/60000]
loss: 1.905776  [ 7600/60000]
loss: 1.784188  [ 8000/60000]
loss: 0.841141  [ 8400/60000]
loss: 1.373933  [ 8800/60000]
loss: 1.378284  [ 9200/60000]
loss: 1.354561  [ 9600/60000]
loss: 0.744157  [10000/60000]
loss: 0.658295  [10400/60000]
loss: 0.989763  [10800/60000]
loss: 1.366536  [11200/60000]
loss: 1.038715  [11600/60000]
loss: 1.192597  [12000/60000]
loss: 1.664562  [12400/60000]
